In [1]:
using JLD

In [2]:
# tic()

In [3]:
# Определение параметров дискретизации

c = 299792458; #Скорость света [м/с]
# c = 3e8; #Скорость света [м/с]
frequency = 2.4e9; # Базовая частота [Гц]
wavelength = c/frequency; # Длина волны в вакууме [м]

# стекло
eps_media=4.7
mu_media=0.999987
sig_media=1e-11

# медь
# eps_media=1.
# mu_media=0.999994
# sig_media=59.5e6


n_coef=sqrt(eps_media*mu_media)

wavelength_in_media = c/n_coef/frequency; # Длина волны в вакууме [м]

@printf "f = %.3f MHz - frequency\n" frequency/10^6;
@printf "λ = %.3f m - wavelength\n" wavelength;
@printf "λ = %.3f m - wavelength in media\n" wavelength_in_media;

# dz = wavelength/100 # Шаг дискретизации пространства [м]
dz = wavelength_in_media/50 # Шаг дискретизации пространства [м]
dt = dz/c # Шаг дискретизации времени [с], Магическое соотношение

@printf "%.5f mm - space step\n" dz*1000;
@printf "%.3fE6 us - time step\n" dt*10^12;

# nlength = Int(round(10*wavelength/dz)) # Количество точек разбиения пространства
# xSteps = 2000 # Количество точек разбиения пространства
# timeSteps = 5000 # Количество временных шагов

S_domain=2
T_domain=0.02e-6

# @printf "%.3f m - space domain length\n" xSteps*dz;
# @printf "%.4f us - time domain length\n" timeSteps*dt*1000000;
@printf "%.3f m - space domain length\n" S_domain;
@printf "%.4f us - time domain length\n" T_domain*1000000;

xSteps = Int(round(S_domain/dz)) # Количество точек разбиения пространства
timeSteps = Int(round(T_domain/dt)) # Количество временных шагов
@printf "%d - number of space steps\n" xSteps;
@printf "%d - number of time steps\n" timeSteps;


f = 2400.000 MHz - frequency
λ = 0.125 m - wavelength
λ = 0.125 m - wavelength in media
2.49828 mm - space step
8.333E6 us - time step
2.000 m - space domain length
0.0200 us - time domain length
801 - number of space steps
2400 - number of time steps


In [4]:
mu0 = 4*pi*10.0^-7; # Магнитная проницаемость
eps0 = 1/(mu0*c*c); # Диэлектрическая проницаемость
# Инициализация массивов проводимости, диэлектрической и магнитной проницаемости
sigma  = zeros(xSteps);
epsilon = ones(xSteps);
mu = ones(xSteps);
# Инициализация массивов коэффициентов
Ca = zeros(xSteps);
Cb = zeros(xSteps);
Da = zeros(xSteps);
Db = zeros(xSteps);
# Инициализация массивов полей
Ex=zeros(timeSteps,xSteps)
Hy=zeros(timeSteps,xSteps)
# Пространсвенная ось
# x = linspace(0,wavelength*10,nlength)
x = linspace(dz,xSteps*dz,xSteps)
# Ось времени
t= linspace(dt,timeSteps*dt,timeSteps);

In [5]:
# Начальные условия
Nu, delta = x[end]/4, wavelength/4 #Переменные ню и дельта для начального распределения Гаусса
G=exp(-((x-Nu).^2)/(2*delta^2)) #Начальное распределение Гаусса
# Параметры среды
left_bord_ind=Int(round(length(x)/2))
right_bord_ind=Int(round(length(x)/4*3));
# left_bord_ind=1;
# right_bord_ind=length(x);
sigma[left_bord_ind:right_bord_ind]=sig_media;
epsilon[left_bord_ind:right_bord_ind]=eps_media;
# mu[left_bord_ind:right_bord_ind]=12.6;
mu[left_bord_ind:right_bord_ind]=mu_media;

# Ex[1,:]=G;
# Hy[1,:]=1/120pi*G;

In [6]:
function signal_gen(t,f)
    signal=cos(2*pi*f/50.*(t-dt)).*sin(2*pi*f.*(t-dt))
#     signal=sin(2*pi*f.*(t-dt))
    return signal
end

signal_gen (generic function with 1 method)

In [7]:
# signal=zeros(t)
signal=signal_gen(t,frequency);
signal_pos=Int(round(length(x)/4))

200

In [8]:
Ca=(1-((sigma*dt)./(2*eps0*epsilon)))./(1+((sigma*dt)./(2*eps0*epsilon)))
Cb = (dt./(eps0*dz*epsilon))./(1+((sigma*dt)./(2*eps0*epsilon)))
Da = (1-sigma*dt./2mu)./(1+sigma*dt./(2*mu0*mu))
Db = (dt./(mu*mu0*dz))./(1+sigma*dt./(2*mu0*mu))

# Ex[1,Int(round(length(x)/4))]+=signal_gen(t[1],frequency)
# Ex[1,Int(round(length(x)/4))+1]+=signal_gen(t[1],frequency)
Ex[1,Int(round(length(x)/4))]+=signal[1]
Ex[1,Int(round(length(x)/4))+1]+=signal[1]

for n = 2:timeSteps #Расчет
    Ex[n,1] = Ex[n-1,2] + ((c*dt-dz)/(c*dt+dz))*(Ex[n,2]-Ex[n-1,1]) #Левая граница в настоящий момент времени. Граничные условия ABC
    Ex[n,2:xSteps] = Ca[2:xSteps].*Ex[n-1,2:xSteps] - Cb[2:xSteps].*(Hy[n-1,2:xSteps]-Hy[n-1,1:xSteps-1]) #Расчет Ex в данный момент времени по всему пространству. Зависит от Hy в прошлый половинный момент времени
    
#     Ex[n,Int(round(length(x)/4))]+=signal_gen(t[n],frequency)
#     Ex[n,Int(round(length(x)/4))+1]+=signal_gen(t[n],frequency)
    Ex[n,signal_pos]+=signal[n]
    Ex[n,signal_pos+1]+=signal[n]
    
    Hy[n,1:xSteps-1] = Da[1:xSteps-1].*Hy[n-1,1:xSteps-1] - Db[1:xSteps-1].*(Ex[n,2:xSteps]-Ex[n,1:xSteps-1]) #Расчет Hy в данный момент времени по всему пространству
    Hy[n,xSteps] = Hy[n-1,xSteps-1] + ((c*dt-dz)/(c*dt+dz))*(Hy[n,xSteps-1]-Hy[n-1,xSteps]) #Правая граница в настоящий момент времени. Граничные условия -- ABC
end

# toc()


In [9]:
save("data.jld", "Ex", Ex, "Hy", Hy, "dt", dt, "dz", dz, "G", G, "x", collect(x), "t", collect(t),"signal",signal,"signal_pos",signal_pos,"xSteps", xSteps, "timeSteps", timeSteps,"left_bord_ind",left_bord_ind,"right_bord_ind",right_bord_ind)